In [1]:
from gezi.common import *
sys.path.append('..')
from src.postprocess import *
from src.config import *
from src.eval import *
from src.ensemble_conf import mns, get_weight

[06/24/22 17:50:17] ensemble_conf.py:131 in <module>
                    list(zip(mns, weights)): [('deberta-v3.lr-2e-5.x', 0.6),
                                              ('patent.rnn_double_dim.x', 0.3),
                                              ('electra-squad.x', 0.2),
                                              ('simcse-patent.rnn_double_dim.x', 0.2),
                                              ('deberta-v3.lr-2e-5.context_key-sector.x', 0.4),
                                              ('patent.rnn_double_dim.context_key-sector.x', 0.1),
                                              ('electra-squad.context_key-sector.x', 0.1)]
                    len(mns): 7


In [2]:
gezi.init_flags()
df = pd.read_csv(f'{FLAGS.root}/train.csv')
df

,id,anchor,target,context,score
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75
2,36d72442aefd8232,abatement,active catalyst,A47,0.25
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00
...,...,...,...,...,...
36468,8e1386cbefd7f245,wood article,wooden article,B44,1.00
36469,42d9e032d1cd3242,wood article,wooden box,B44,0.50
36470,208654ccb9e14fa3,wood article,wooden handle,B44,0.50
36471,756ec035e694722b,wood article,wooden material,B44,0.75


In [3]:
FOLDS = 5
xs = []
ensemblers = [gezi.Ensembler(need_sort=True, includes=['pred']) for _ in range(FOLDS)]
for model in mns:
  for fold in range(FOLDS):
    root = f'../working/offline/10/{fold}/{model}'
    x = gezi.load(f'{root}/valid.pkl')
    x['fold'] = np.asarray([fold] * len(x['id']))
    x['pred'] = normalize(x['pred'])
    # xs.append(x)
    ensemblers[fold].add(x, get_weight(model))
xs = [ensembler.finalize() for ensembler in ensemblers]
for x in xs:
  x['pred'] = normalize(x['pred'])
x = gezi.merge_array_dicts(xs)
x

{'id': array(['00068e3dfd5515df', '000c8495ef1763e1', '0019b063a0fe1dc8', ...,
        'ffb6e883de7d9d67', 'fff0b4d9fca7fae3', 'fff8d1ccce8e20ec'],
       dtype='<U16'),
 'label': array([0.25, 0.  , 0.  , ..., 0.5 , 0.25, 0.  ]),
 'anchor': array(['conductor particles', 'page file', 'predetermined acceleration',
        ..., 'water propulsion device',
        'high gradient magnetic separators', 'perform working operations'],
       dtype='<U38'),
 'target': array(['resin cores', 'hidden camera', 'predetermined policy', ...,
        'propulsion unit', 'magnetic equipment',
        'psychology of working framework'], dtype='<U98'),
 'text': array(['conductor particles[SEP]resin cores[SEP]PERFORMING OPERATIONS; TRANSPORTING. PHYSICAL OR CHEMICAL PROCESSES OR APPARATUS IN GENERAL[SEP]metal; conducting particles; gold tooth; transparent spheres particles; ballpen ink; sources; exchange membranes; electro conductive powders; source membranes; conductor fine particles; source electrodes; con

In [4]:
oof = pd.DataFrame(x)

In [5]:
calc_metric(oof.label, oof.pred)

0.8682403177522899